# 4. How do measurable/computable performance features relate to the expressive character dimensions?

In this notebook we study whether there is a systematic relationship between the expressive character dimensions (Section 5 in the paper, notebook `02_dimensions_for_expressive_character.ipynb`) and measurable or computed performance qualities that can be extracted directly from the audio files or from the score-to-performance alignments.

In the rest of this notebook, we refer to these measurable or computed performance qualities as **performance features**. We will consider three classes of performance features:

1. Performance Parameters
2. Mid-level features
3. High-level features

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
# from scripts.performance_analysis import load_performance
from stat_utils import (LinearModel, 
                        cod, 
                        standardize, 
                        zheng_loh_model_selection, 
                        parameter_stats, 
                        correlation,
                        ttest_ind,
                        pretty_print_results)
# from scripts.utils import PIECE_NAMES, SCORE_NAMES, list_files_deep, get_file_from_id
import partitura
# from scipy.stats import kurtosis, skew

We start by loading the embeddings of the performances (i.e., their centroid) in the 4D PCA space identified in `02_dimensions_for_expressive_character.ipynb`. For convenience, we include the embeddings of each piece in a csv file.

In [ ]:
# Load the embeddings
df_embeddings = pd.read_csv('./perf_embeddings/perf_embeddings_final.csv')

# Name of the pieces
piece_names = np.unique(df_embeddings['piece_name'])

# Names of the expressive parameters (to initialize the columns in df_embeddings)
perf_cols = ['beat_period', 
             'loudness']

stat_names = ['_mean', '_std', '_kurtosis', '_skewness']
for col in perf_cols:
    
    if col != 'cresc_factor':
        for st in stat_names:
            df_embeddings[col + st] = np.zeros(len(df_embeddings))
    else:
        df_embeddings[col] = np.zeros(len(df_embeddings))

all_music_ids = df_embeddings.music_id.values

mid_idxs = np.array([int(np.where(all_music_ids == mi)[0]) for mi in all_music_ids])

In [ ]:
alignments_dir = './scores/ApproximateMIDI/match'
scores_dir = './scores/MusicXML'
loudness_dir = './beat_annotations/Loudness'

alignments_list = sorted(list_files_deep(alignments_dir, full_paths=True))
loudness_list = sorted(list_files_deep(loudness_dir, full_paths=True))
scores_list = sorted(list_files_deep(scores_dir, full_paths=True))

In [ ]:
perf_dataframes = dict()
for i, piece_name in enumerate(piece_names):
    spart = partitura.load_musicxml(os.path.join(scores_dir, SCORE_NAMES[piece_name] + '.musicxml'))
    # df_perf = load_performance()
    music_ids = df_embeddings[df_embeddings.piece_name==piece_name]['music_id'].values
        
    for mid in music_ids:
        print('loading {0}'.format(mid))
        match_fn = get_file_from_id(alignments_list, mid)
        loudness_fn = get_file_from_id(loudness_list, mid)
        df, beat_map, inv_beat_map = load_performance(spart_or_xml_fn=spart,
                                                      match_fn=match_fn,
                                                      loudness_fn=loudness_fn)
        perf_dataframes[mid] = df


In [ ]:
for col in perf_cols:
    for mi, mix in zip(all_music_ids, mid_idxs):
        if col != 'cresc_factor':
            for stn, st in zip(stat_names, parameter_stats(perf_dataframes[mi][col].values)):
                df_embeddings[col + stn][mix] = st
        else:
            df_embeddings[col][mix] = perf_dataframes[mi][col].values.mean()


## Performance Parameters

We consider two performance parameters, **tempo** and **dynamics**, to relate to the expressive dimensions described in the previous notebook.

1. The *tempo curve* are extracted directly from the hand-corrected score-to-performance alignments by computing the inter-beat intervals (i.e., **beat period** in seconds per beat)

2. We extracted *loudness curves* from the audio files using a perceptually weighted smoothing of the signal energy (thanks to Olivier Lartillot).

For inter- and intra-piece comparisons, we calculate the average value, standard deviation, kurtosis and skewness of these curves:

* *Average beat period*: how fast a performance is (a larger average beat period means a slower performance)
* *Average loudness*: how loud a performance is (a larger loudness means a louder performance)


In [ ]:
input_cols = []
for cn in perf_cols:
    if cn != 'cresc_factor':
        input_cols += [cn + st for st in stat_names]
    else:
        input_cols.append(cn)

target_cols = ['dim_{0}'.format(i) for i in range(1, 5)]

xy = df_embeddings[input_cols + target_cols].dropna()
x = standardize(xy[input_cols].values)
# x = xy[input_cols].values
for target in target_cols:
    y = standardize(xy[target].values)
    # y = xy[target].values
    lm, bp_ix = zheng_loh_model_selection(x=x,
                                          y=y,
                                          input_names=input_cols)
    statistics, ttest, y_hat = lm.test(x=x[:, bp_ix], 
                                y=y,
                                return_preds=True, 
                                significance_level=0.05)
    
    ttest
    corrs = [correlation(y, sl * x[:, bi]) for bi, sl in zip(bp_ix, lm.params)]
    print(target, cod(y, y_hat, p=len(bp_ix)))
    for sl, st, cr in zip(lm.params, statistics, corrs):
        print(st.parameter_name, sl, st.pvalue, 
              st.reject_h0, cr,'\n')

## Mid-level Features

These features are perceptual qualities of music such as *articulation*, *rhytmic clarity* and *modality* that describe overall properties of musical exceprts and are intuitively clear to listeners 


In [ ]:
mid_level_cols = ['melody', 'articulation', 'rhythm_complexity', 
                  'rhythm_stability', 'dissonance', 'atonality', 'mode']

input_cols = mid_level_cols
target_cols = ['dim_{0}'.format(i) for i in range(1, 5)]

xy = df_embeddings[input_cols + target_cols].dropna()
x = standardize(xy[input_cols].values)
# x = xy[input_cols].values
for target in target_cols:
    y = standardize(xy[target].values)
    # y = xy[target].values
    lm, bp_ix = zheng_loh_model_selection(x=x,
                                          y=y,
                                          input_names=input_cols)
    statistics, ttest, y_hat = lm.test(x=x[:, bp_ix], 
                                y=y,
                                return_preds=True, 
                                significance_level=0.05)
    corrs = [correlation(y, sl * x[:, bi]) for bi, sl in zip(bp_ix, lm.params)]
    print(target, cod(y, y_hat, p=len(bp_ix)))
    for sl, st, cr in zip(lm.params, statistics, corrs):
        print(st.parameter_name, sl, st.pvalue, 
              st.reject_h0, cr,'\n')

## Arousal valence (Non domain adapted)

In [ ]:
# Load the embeddings
# df_embeddings = pd.read_csv('./perf_embeddings/perf_embeddings_w_performance.csv')

# Name of the pieces
# piece_names = np.unique(df_embeddings['piece_name'])

# # Names of the expressive parameters (to initialize the columns in df_embeddings)
# perf_cols = ['beat_period', 'articulation_log', 'velocity_trend', 
#              'velocity_dev', 'majorness', 'loudness', 'cresc_factor']

# # for col in perf_cols:
# #     df_embeddings[col] = np.zeros(len(df_embeddings))

# all_music_ids = df_embeddings.music_id.values

# mid_idxs = np.array([int(np.where(all_music_ids == mi)[0]) for mi in all_music_ids])


In [ ]:
# Extract statistical features from dynamic arousal valence
all_avs = list_files_deep('./audio_extracted_data/avs_3f5df_mls_1479/')
row_header = ['music_id', 'a_max', 'v_max',
              'a_min', 'v_min',
              'a_std', 'v_std',
              'a_avg', 'v_avg',
              'a_med', 'v_med',
             'a_kurtosis', 'v_kurtosis',
             'a_skewness', 'v_skewness']
vals = []

for i in all_avs:
    music_id = int(i[:2])
    a, v = np.load(os.path.join('./audio_extracted_data/avs_3f5df_mls_1479', i))
    a_max, v_max = max(a), max(v)
    a_min, v_min = min(a), min(v)
    a_std, v_std = np.std(a), np.std(v)
    a_avg, v_avg = np.mean(a), np.mean(v)
    a_med, v_med = np.median(a), np.median(v)
    a_kurtosis, v_kurtosis = kurtosis(a), kurtosis(v)
    a_skewness, v_skewness = skew(a), skew(v)
    vals.append([music_id, a_max, v_max,
                 a_min, v_min,
                 a_std, v_std,
                 a_avg, v_avg,
                 a_med, v_med,
                 a_kurtosis, v_kurtosis,
                 a_skewness, v_skewness])
    
vals = np.vstack(vals)
df_av = pd.DataFrame(data=vals, columns=row_header)


In [ ]:
df_av.music_id = df_av.music_id.astype(int)
df_av.mean()

In [ ]:
df_embeddings = df_embeddings.set_index('music_id')
df_av = df_av.set_index('music_id')
df = pd.concat([df_embeddings, df_av], axis=1)
df.head()

In [ ]:
df = pd.concat([df_embeddings, df_av], axis=1)

In [ ]:
input_cols = [# 'a_max', 'v_max',
              # 'a_min', 'v_min',
              'a_std', 'v_std',
              'a_avg', 'v_avg',
              # 'a_med', 'v_med',
        'a_kurtosis', 'v_kurtosis',
    'a_skewness', 'v_skewness'
             ]

target_cols = ['dim_{0}'.format(i) for i in range(1, 5)]

xy = df[input_cols + target_cols].dropna()
x = standardize(xy[input_cols].values)
for target in target_cols:
    y = standardize(xy[target].values)
    lm, bp_ix = zheng_loh_model_selection(x=x,
                                          y=y,
                                          input_names=input_cols)
    # lm = LinearModel(input_names=input_cols)
    statistics, ttest, y_hat = lm.test(x=x[:, bp_ix], 
                                y=y,
                                return_preds=True, 
                                significance_level=0.05)
    corrs = [correlation(y, sl * x[:, bi]) for bi, sl in zip(bp_ix, lm.params)]
    print(target, cod(y, y_hat, p=len(bp_ix)))
    for sl, st, cr in zip(lm.params, statistics, corrs):
        print(st.parameter_name, sl, st.pvalue, 
              st.reject_h0, cr,'\n')

## Arousal valence (domain adapted)

In [ ]:
# Extract statistical features from dynamic arousal valence
all_avs = list_files_deep('./audio_extracted_data/avs_5490a_mls_c5612//')
row_header = ['music_id', 'a_max', 'v_max',
                            'a_min', 'v_min',
                            'a_std', 'v_std',
                            'a_avg', 'v_avg',
                            'a_med', 'v_med',
             'a_kurtosis', 'v_kurtosis',
             'a_skewness', 'v_skewness'
             ]
vals = []

for i in all_avs:
    music_id = int(i[:2])
    a, v = np.load(os.path.join('./audio_extracted_data/avs_5490a_mls_c5612', i))
    a_max, v_max = max(a), max(v)
    a_min, v_min = min(a), min(v)
    a_std, v_std = np.std(a), np.std(v)
    a_avg, v_avg = np.mean(a), np.mean(v)
    a_med, v_med = np.median(a), np.median(v)
    a_kurtosis, v_kurtosis = kurtosis(a), kurtosis(v)
    a_skewness, v_skewness = skew(a), skew(v)
    vals.append([music_id, a_max, v_max,
                    a_min, v_min,
                    a_std, v_std,
                    a_avg, v_avg,
                    a_med, v_med,
                a_kurtosis, v_kurtosis,
                a_skewness, v_skewness])
    
vals = np.vstack(vals)
df_av = pd.DataFrame(data=vals, columns=row_header)

In [ ]:
df_av.music_id = df_av.music_id.astype(int)
df_av.mean()

In [ ]:
df_av = df_av.set_index('music_id')
df = pd.concat([df_embeddings, df_av], axis=1)
df.head()

In [ ]:
df = pd.concat([df_embeddings, df_av], axis=1)


In [ ]:
input_cols = [# 'a_max', 'v_max',
              # 'a_min', 'v_min',
              'a_std', 'v_std',
              'a_avg', 'v_avg',
              # 'a_med', 'v_med',
    'a_kurtosis', 'v_kurtosis',
    'a_skewness', 'v_skewness'
             ]

target_cols = ['dim_{0}'.format(i) for i in range(1, 5)]
# target_cols = ['dim_3']

xy = df[input_cols + target_cols].dropna()
x = standardize(xy[input_cols].values)
for target in target_cols:
    y = standardize(xy[target].values)
    lm, bp_ix = zheng_loh_model_selection(x=x,
                                          y=y,
                                          input_names=input_cols)
    statistics, ttest, y_hat = lm.test(x=x[:, bp_ix], 
                                y=y,
                                return_preds=True, 
                                significance_level=0.05)
    corrs = [correlation(y, sl * x[:, bi]) for bi, sl in zip(bp_ix, lm.params)]
    print(target, cod(y, y_hat, p=len(bp_ix)))
    for sl, st, cr in zip(lm.params, statistics, corrs):
        print(st.parameter_name, sl, st.pvalue, 
              st.reject_h0, cr,'\n')

In [ ]:
from sklearn.model_selection import train_test_split, KFold

kf = KFold(n_splits=len(y), random_state=1984, shuffle=True)

tests = []
for train_ix, test_ix in kf.split(x):
    
    lm = LinearModel()
    lm.fit_predict(x[train_ix][:, bp_ix], y[train_ix])
    
    y_hat = lm.predict(x[test_ix][:, bp_ix])
    tests.append(np.column_stack((y_hat, y[test_ix])))
    
tests = np.vstack(tests)

print(cod(tests[:, 1], tests[:, 0]))